In [1]:
!nvidia-smi

Sun Oct 27 04:06:00 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM3-32GB           Off |   00000000:05:00.0 Off |                    0 |
| N/A   35C    P0             63W /  350W |   30825MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModel, Trainer, TrainingArguments, BertForMaskedLM, AutoModelForCausalLM
from transformers import AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm.notebook import tqdm, trange
import numpy as np
from scipy.spatial.distance import cosine
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GenerationConfig
import os
import json
from peft import PeftModel, PeftConfig
from pathlib import Path

from text import collect_text_blocks

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
torch.cuda.empty_cache()

## Trying xlm-robert-by

моделька от сбера обученная специально под задачи qna, сильно не впечатлила, пришлось отказаться

In [49]:
path_to_file = Path("book1.docx")
blocks = collect_text_blocks([path_to_file])

Getting text blocks from book1.docx


In [50]:
blocks = " ".join(blocks)

In [51]:
qa_model = AutoModelForQuestionAnswering.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru").to(device)
qa_tokenizer = AutoTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")
qa_pipeline = pipeline("question-answering", model="AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [59]:
questions = [
    "С какой целью проводится инвентаризация источников выбросов вредных веществ?",
    "Сколько видов деятельности осуществляется АО «Кирпич»?",
    "По какому адресу располагается производственная площадка?",
    "Какой кадастровый номер присвоен участку?",
    "На каком расстоянии от границ площадки располагается особая зона?",
    "Какое разрешенное использование у нормируемой территории?",
    "На каком расстоянии от границы площадки расположена ближайшая нормируемая территория?",
    "Какой размер нормативной СЗЗ, согласно требованиям СаНПиН, определен для производственной площадки? ",
    "Перечислить этапы добычи на карьере",
    "Сколько загрязняющих веществ попадает в атмосферу? Перечислить их.",
    "Сколько методических документов было использовано для того, чтобы выполнить расчет ИЗАВ №6001?",
    "На сколько % должны превышать показатели выбросов конкретного источника выбросов по конкретному веществу соответствующие максимальные разовые показатели, чтобы провести корректировку данных инвентаризации выбросов? "
]

In [83]:
def get_best_answer_from_large_text(question, context_chunk):
    inputs = qa_tokenizer(question + " Объясни максимально подробно, это очень важно", context_chunk, max_length=512, stride=128, )
    result = qa_model(**inputs)
    
    return result

In [84]:
for question in questions:
    print(get_best_answer_from_large_text(question, blocks))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


AttributeError: 'list' object has no attribute 'size'

##  Domain adaptation bert

Данные весьма специфичны и узко направлены, значит надо дать моделям необходимую базу знаний о происходящем. Поэтому мы дообучаем в данном случае rubert-large на данных нам книгах, пособиях и сборниках законов в виде сплошного текста

In [5]:
model_name = "ai-forever/ruBert-large"

In [6]:
model = BertForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at ai-forever/ruBert-large were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weigh

In [7]:
data_path = "blocks-1.json"

with open(data_path) as f:
    data = json.load(f)

In [8]:
len(data)

18294

In [9]:
data = " ".join(data)

делим непрерывный текст на чанки с пересечением

In [10]:
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=512,
    chunk_overlap=64
)

In [11]:
chunks = text_splitter.split_text(data)
chunks

['Отчет по инвентаризации выбросов вредных (загрязняющих) веществ в атмосферный воздух и их источников промплощадки Сельскохозяйственного Производственного Кооператива «Рассвет» (СПК «Рассвет»), расположенной по адресу: Ростовская область, Егорлыкский район, составлен на 20 страницах. Результаты проведенных работ по инвентаризации оформлены в виде таблиц, представленных в Приложении 1 данного отчета. Качественные и количественные характеристики выбросов определены расчетными методами по методикам, рекомендованным [1], а также с использованием программных комплексов фирмы «ИНТЕГРАЛ». Федеральный закон Российской Федерации от 4 мая 1999 года №96-ФЗ «Об охране атмосферного воздуха» [2]; Постановление Правительства РФ от 03.08.92 г. № 545 «Порядок',
 'воздуха» [2]; Постановление Правительства РФ от 03.08.92 г. № 545 «Порядок разработки и утверждения экологических нормативов выбросов загрязняющих веществ, лимитов использования природных ресурсов, размещения отходов» [3]; Нормативно-техничес

In [12]:
inputs = tokenizer(
    chunks,
    max_length=512,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

In [13]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
inputs["labels"] = inputs["input_ids"].detach().clone()

In [15]:
random_tensor = torch.randn(inputs["input_ids"].shape)

In [16]:
random_tensor.shape

torch.Size([8673, 512])

обучаем нашу модель под задачу заполнения пустых частей маски с 15% вероятность

In [17]:
masked_tensor = (random_tensor < 0.15)*(inputs['input_ids'] != 101)*(inputs['input_ids'] != 102)*(inputs['input_ids'] != 0)

In [18]:
nonzeros_indices = []
for i in range(len(masked_tensor)):
    nonzeros_indices.append(torch.flatten(masked_tensor[i].nonzero()).tolist())

In [19]:
for i in range(len(inputs['input_ids'])):
    inputs['input_ids'][i, nonzeros_indices[i]] = 103

In [20]:
class BookDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        labels = self.encodings['labels'][index]
        attention_mask = self.encodings['attention_mask'][index]
        token_type_ids = self.encodings['token_type_ids'][index]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'token_type_ids': token_type_ids
        }

In [21]:
dataset = BookDataset(inputs)

In [22]:
dataloader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True
)

In [23]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e

In [24]:
epochs = 3
optimizer = AdamW(model.parameters(), lr=1e-5)

/home/user1/environments/hack/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


собственно, процесс обучения. Было выбрано 3 эпохи, так как дальнейшее обучение смысло не имеет

In [26]:
model.train()

losses = []

for epoch in range(epochs):
    total_loss = 0.0
    
    loop = tqdm(dataloader)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()

        loop.set_description("Epoch: {}".format(epoch))
        loop.set_postfix(loss=loss.item())

    losses.append(total_loss / len(dataloader))

  0%|          | 0/1085 [00:00<?, ?it/s]

  0%|          | 0/1085 [00:00<?, ?it/s]

  0%|          | 0/1085 [00:00<?, ?it/s]

значения функции потерь после 3 эпох:

In [27]:
losses

[0.6220568792885899, 0.5518541081709796, 0.5007469304970332]

сохраняем получившиеся модели в соответствующие файлы:

In [28]:
output_models_dir = "models2"

model.save_pretrained(output_models_dir)
tokenizer.save_pretrained(output_models_dir)

('models2/tokenizer_config.json',
 'models2/special_tokens_map.json',
 'models2/vocab.txt',
 'models2/added_tokens.json',
 'models2/tokenizer.json')

## Feature Extraction

Для представления текста в виде эмбеддингов нам необходима функция которая будет отбирать фичи. В нашем случае у нас есть обученная на специфияческом наборе данных модель берта, которую можно использовать для представления данных, если доставать последний скрытый слой

In [5]:
model = BertForMaskedLM.from_pretrained("models2")
tokenizer = AutoTokenizer.from_pretrained("models2")
model.to(device)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e

In [31]:
torch.cuda.empty_cache()

Был выбран метод взятия эмбеддинга по среднему

In [6]:
def extract_features(text):
    model.eval()

    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=512,
        chunk_overlap=64
    )

    chunks = text_splitter.split_text(text)

    features = []

    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]

        attention_mask = inputs['attention_mask']
        mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())
        sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
        mean_embeddings = sum_embeddings / torch.clamp(mask_expanded.sum(1), min=1e-9)
        
        features.append(mean_embeddings)

    return torch.stack(features).mean(dim=0).squeeze(0).cpu()    

В дальнейшем схожесть двух элементов будем оценивать по косинусной схожести - в этом плане использование векторного представления очень удобно.

In [7]:
def get_cosine_similarity(vector_1, vector_2):
    return 1 - cosine(vector_1, vector_2)

In [8]:
emb1 = extract_features("какие штрафы назначены за нарушение законов ук рф в области природоохраны?")
emb2 = extract_features("законом предусмотрено до 5 лет лишения свободы за серьезные нарушени в области экологии")

print(get_cosine_similarity(emb1, emb2))

0.7856023933870414


## Using LLM Saiga

Для критической обработки даных которые пришли из контекста и их суммаризации мы использовали ЛЛМ Сайга с низкой температурой.
Был и другой вариант обрабатывать контекст, о нем в ноутбуке training model based on...

In [12]:
MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.temperature = 0.2
generation_config.frequency_penalty = 1.7

print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "frequency_penalty": 1.7,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [33]:
task = "В прошлом январе среднестатистическая температура была в районе 30 градусов. минимальная температура прошлой зимой была -30. летом было тепло. Посмотри на данные, что были до и ответь: какая минимальная температура была зимой? Ответь только на этот вопрос и смотри очень внимательно. Если ничего не найдено, ответь, что нет данных"

## Data generation for RAG DB

Неотъемлемая часть хорошего RAG - память. Далее мы из текста учебников, пособий и сводок создаём эмбеддинги, чтобы положить их в базу и сравнивать с каждым запросом приходяшим в базу

In [36]:
data_path = "blocks-1.json"

with open(data_path) as f:
    data = f.read().replace("\n", " ")

In [37]:
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=512,
    chunk_overlap=64
)

In [38]:
chunks = text_splitter.split_text(data)

In [39]:
embeddings = {}

for i in trange(len(chunks)):
    embeddings[i] = extract_features(chunks[i])

  0%|          | 0/9435 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [50]:
for i in trange(len(chunks)):
    torch.save(embeddings[i], f"vector_{i}.pt")
    with open(f"text_{i}.txt", encoding="utf8", mode="w") as f:
        f.write(chun

tensor([ 0.1473, -0.1392,  0.1913,  ...,  0.2460,  0.5051,  0.4014])

## QDrant DB for RAG

В нашем решении мы используем векторную базу QDrant для быстрого поиска схожих элементов

In [13]:
from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

from json import load

class CustomEmbedding(Embeddings):
    def __init__(self, directory: str = "models"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = BertForMaskedLM.from_pretrained(directory)
        self.tokenizer = AutoTokenizer.from_pretrained(directory)
        self.model.to(self.device)
    
    def embed_query(self, text: str) -> list[float]:
        return extract_features(text).tolist()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return list(map(lambda text: self.embed_query(text), texts))


COLLECTION_NAME = "upload_test_collection"
SIZE = 1024
QDRANT_URL = "http://213.171.5.51:6333"

client = QdrantClient(url=QDRANT_URL)

embedding = CustomEmbedding()

print(isinstance(embedding.embed_query("Шлёпа"), list))

True


In [15]:
# with open("./blocks.json", mode="r") as text_block_file:
#     blocks = list(filter(lambda block: len(block) >= 70, load(text_block_file)))
    
# blocks = blocks[:500]

qdrant = QdrantVectorStore(
    vectors_config=VectorParams(size=SIZE, distance=Distance.COSINE),
    client=client,
    collection_name="Cancer test",
    embedding=embedding,
)


TypeError: QdrantVectorStore.__init__() got an unexpected keyword argument 'vectors_config'

In [16]:
print([i.page_content for i in qdrant.similarity_search(question, k=10)])

NameError: name 'qdrant' is not defined